# 1.Creat PostgreSQL database with 4 tables: Customer, Account, Alert, Transaction

In [1]:
!sudo apt install -y pipx


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pipx is already the newest version (1.4.3-1).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.


In [2]:
!pipx ensurepath

/home/ubuntu/.local/bin is already in PATH.

⚠️  All pipx binary directories have been added to PATH. If you are sure you
want to proceed, try again with the '--force' flag.

Otherwise pipx is ready to go! ✨ 🌟 ✨


In [3]:
!sudo apt install -y python3-psycopg2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-psycopg2 is already the newest version (2.9.9-1build1).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.


### Creat Database

In [4]:
# Run the following command to ensure that PostgreSQL is installed
!sudo apt update
!sudo apt install -y postgresql postgresql-contrib


Hit:1 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble InRelease
Get:2 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:3 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:4 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [916 kB]
Get:5 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [151 kB]
Get:6 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Packages [1040 kB]
Get:7 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Components [364 kB]
Get:8 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/restricted amd64 Components [212 B]
Get:9 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/multiverse amd64 Components [940 B]
Get:10 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-backports/main amd64 Components [208 B]
Get:11 http://us-east-1.ec2.archive.ubuntu.com/ubuntu 

In [5]:
# Start PostgreSQL
!sudo systemctl start postgresql
!ss -tulnp | grep 5432

tcp   LISTEN 0      200            127.0.0.1:5432       0.0.0.0:*                                             


In [6]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Connect to the PostgreSQL default database
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="yourpassword",
    host="localhost",
    port="5432"
)

# **Set ISOLATION_LEVEL_AUTOCOMMIT**
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()

cursor.execute(f"DROP DATABASE IF EXISTS aml_database;")

# **Creating a Database**
cursor.execute("CREATE DATABASE aml_database;")

# Close the connection
cursor.close()
conn.close()

print("Database created successfully!")


Database created successfully!


### Check whether the database is connected normally: SELECT version()

In [7]:
conn = psycopg2.connect(
    dbname="aml_database",  # Connect to the newly created database
    user="postgres",
    password="yourpassword",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Check whether the database is connected normally
cursor.execute("SELECT version();")
print(cursor.fetchone())

# Close the connection
cursor.close()
conn.close()


('PostgreSQL 16.8 (Ubuntu 16.8-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0, 64-bit',)


### Create 4 tables structure in database

In [8]:
# Reconnect to aml_database
conn = psycopg2.connect(
    dbname="aml_database",
    user="postgres",
    password="yourpassword",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# SQL Statements
create_tables_query = '''
DROP TABLE IF EXISTS Transaction;
DROP TABLE IF EXISTS Alert;
DROP TABLE IF EXISTS Account;
DROP TABLE IF EXISTS Customer;

CREATE TABLE Customer (
    customer_id VARCHAR(50) PRIMARY KEY,
    customer_type VARCHAR(50) NOT NULL,
    customer_name VARCHAR(100) NOT NULL,
    customer_line_of_business VARCHAR(100),
    customer_expected_products TEXT,
    customer_expected_geographies TEXT,
    customer_incorporation_residence_country VARCHAR(10) NOT NULL
);

CREATE TABLE Account (
    account_id VARCHAR(50) PRIMARY KEY,
    customer_id VARCHAR(50) REFERENCES Customer(customer_id) ON DELETE CASCADE,
    date_of_opening DATE NOT NULL,
    expected_incoming_activity NUMERIC(15,2),
    expected_outgoing_activity NUMERIC(15,2)
);

CREATE TABLE Alert (
    detection_id VARCHAR(50) PRIMARY KEY,
    alert_id VARCHAR(50) NOT NULL,
    alert_date DATE NOT NULL,
    customer_id VARCHAR(50) REFERENCES Customer(customer_id) ON DELETE SET NULL,
    rule_name VARCHAR(255) NOT NULL,
    alerted_transactions TEXT,
    false_positive_true_positive VARCHAR(20),
    alert_narrative TEXT
);

CREATE TABLE Transaction (
    transaction_id VARCHAR(50) PRIMARY KEY,
    transaction_date DATE NOT NULL,
    transaction_type VARCHAR(100) NOT NULL,
    customer_id VARCHAR(50) REFERENCES Customer(customer_id) ON DELETE SET NULL,
    account_id VARCHAR(50) REFERENCES Account(account_id) ON DELETE CASCADE,
    incoming_outgoing VARCHAR(20) CHECK (incoming_outgoing IN ('Incoming', 'Outgoing')),
    amount NUMERIC(15,2) NOT NULL,
    originator VARCHAR(100),
    originator_country VARCHAR(10),
    beneficiary VARCHAR(100),
    beneficiary_country VARCHAR(10)
);
'''

# Execute SQL statements
cursor.execute(create_tables_query)
conn.commit()

print("All tables created successfully!")

# Close the connection
cursor.close()
conn.close()


All tables created successfully!


### Import four local CSV files into the database

In [9]:
!sudo apt install -y python3-pandas

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pandas is already the newest version (2.1.4+dfsg-7).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.


In [10]:
import pandas as pd

# Reestablishing a connection
conn = psycopg2.connect(
    dbname="aml_database",
    user="postgres",
    password="yourpassword",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# ------------------- 1. Import Customer table ------------------- #
print("Starting import of Customer data...")
customer_df = pd.read_csv("Customer Table.csv")
# Remove duplicate customer_id
customer_df = customer_df.drop_duplicates(subset=["Customer ID"])

for _, row in customer_df.iterrows():
    cursor.execute(
        "INSERT INTO Customer (customer_id, customer_type, customer_name, customer_line_of_business, customer_expected_products, customer_expected_geographies, customer_incorporation_residence_country) VALUES (%s, %s, %s, %s, %s, %s, %s)",
        (row["Customer ID"], row["Customer Type"], row["Customer Name"], row["Customer Line of Business"], row["Customer Expected Products"], row["Customer Expected Geographies"], row["Customer Incorporation/Residence Country"])
    )

conn.commit()
print("Customer data import completed!")

# ------------------- 2. Import Account table ------------------- #
print("\nStart importing Account data...")
account_df = pd.read_csv("Account Table.csv")
# Remove duplicate account_id
account_df = account_df.drop_duplicates(subset=["Account ID"])

# Import only customer_ids that exist in the Customer table
cursor.execute("SELECT customer_id FROM Customer")
valid_customers = {row[0] for row in cursor.fetchall()}

for _, row in account_df.iterrows():
    if row["Customer ID"] in valid_customers:
        cursor.execute(
            "INSERT INTO Account (account_id, customer_id, date_of_opening, expected_incoming_activity, expected_outgoing_activity) VALUES (%s, %s, DATE '1900-01-01' + INTERVAL '1 day' * %s, %s, %s) ON CONFLICT (account_id) DO NOTHING",
            (row["Account ID"], row["Customer ID"], row["Date of Opening"], row["Expected Incoming Activity"], row["Expected Outgoing Activity"])
        )
conn.commit()
print("Account data import completed!")

# ------------------- 3. Import Alert table ------------------- #
print("\nStart importing Alert data...")
alert_df = pd.read_csv("Alert Table.csv")
# Remove duplicate detection_id (since it is now the primary key)
alert_df = alert_df.drop_duplicates(subset=["Detection ID"])

# customer_id is allowed to be NULL, but if present, it must be in the Customer table
for _, row in alert_df.iterrows():
    customer_id = row["Customer ID"] if pd.notna(row["Customer ID"]) and row["Customer ID"] in valid_customers else None
    cursor.execute(
        "INSERT INTO Alert (detection_id, alert_id, alert_date, customer_id, rule_name, alerted_transactions, false_positive_true_positive, alert_narrative) VALUES (%s, %s, DATE '1900-01-01' + INTERVAL '1 day' * %s, %s, %s, %s, %s, %s) ON CONFLICT (detection_id) DO NOTHING",
        (row["Detection ID"], row["Alert ID"], row["Alert Date"], customer_id, row["Rule Name"], row["Alerted Transactions per Detection"], row["False Positive / True Positive"], row["Alert Narrative"])
    )
conn.commit()
print("Alert data import completed!")

# ------------------- 4. Import Transaction table ------------------- #
print("\nStart importing transaction data...")

transaction_df = pd.read_csv("Transaction Table.csv")
# Remove duplicate transaction_id
transaction_df = transaction_df.drop_duplicates(subset=["Transaction ID"])

# Get a valid customer_id and account_id
cursor.execute("SELECT customer_id FROM Customer")
valid_customers = {row[0] for row in cursor.fetchall()}

cursor.execute("SELECT account_id FROM Account")
valid_accounts = {row[0] for row in cursor.fetchall()}

for _, row in transaction_df.iterrows():
    customer_id = row["Customer ID"] if pd.notna(row["Customer ID"]) and row["Customer ID"] in valid_customers else None
    account_id = row["Account"] if pd.notna(row["Account"]) and row["Account"] in valid_accounts else None

    if account_id is not None:  # account_id is required because it is a foreign key
        # Make sure all None are handled correctly
        transaction_values = (
            row["Transaction ID"],
            row["Transaction Date"],
            row["Transaction Type"],
            customer_id,
            account_id,
            row["Incoming/Outgoing"] if pd.notna(row["Incoming/Outgoing"]) else None,
            row["Amount"] if pd.notna(row["Amount"]) else 0,  # If the numeric column is empty, replace it with 0, or use None as needed
            row["Originator"] if pd.notna(row["Originator"]) else None,
            row["Originator Country"] if pd.notna(row["Originator Country"]) else None,
            row["Beneficiary"] if pd.notna(row["Beneficiary"]) else None,
            row["Beneficiary Country"] if pd.notna(row["Beneficiary Country"]) else None
        )

        cursor.execute(
            "INSERT INTO Transaction (transaction_id, transaction_date, transaction_type, customer_id, account_id, incoming_outgoing, amount, originator, originator_country, beneficiary, beneficiary_country) "
            "VALUES (%s, DATE '1900-01-01' + INTERVAL '1 day' * %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            "ON CONFLICT (transaction_id) DO NOTHING",
            transaction_values
        )

conn.commit()
print("Transaction data import completed!")

# Close the database connection
cursor.close()
conn.close()
print("\nAll data has been successfully imported into the database!")

Starting import of Customer data...
Customer data import completed!

Start importing Account data...
Account data import completed!

Start importing Alert data...
Alert data import completed!

Start importing transaction data...
Transaction data import completed!

All data has been successfully imported into the database!


### Check the four tables in the database to see if they have been successfully imported.

In [11]:
!sudo apt install -y python3-sqlalchemy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-sqlalchemy is already the newest version (1.4.50+ds1-1build1).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.


In [12]:
from sqlalchemy import create_engine
#import pandas as pd

# Creating a SQLAlchemy Engine
db_user = "postgres"
db_password = "yourpassword"  # Replace with your password
db_host = "localhost"
db_name = "aml_database"

engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")


In [13]:
# ------------------- 1. View the Customer table ------------------- #
print("\nViewing data from the Customer table...")
query = "SELECT * FROM Customer LIMIT 10;"
df_customer = pd.read_sql(query, engine)
display(df_customer)



Viewing data from the Customer table...


,customer_id,customer_type,customer_name,customer_line_of_business,customer_expected_products,customer_expected_geographies,customer_incorporation_residence_country
0,C-1,Individual,John Diamond,Manufacturing,ACH; Wire,US,US
1,C-2,Business,RDF Plumbing,Plumbing Services,ACH; Wire; Cash Deposit; Internal Transfer,US,US
2,C-3,Individual,Kyle Strong,Service Industry,ACH; Wire; Cash Deposit; Internal Transfer,US; HK,HK
3,C-4,Business,JDF Industries,Oil refinement,ACH; Wire,US; SA,US


In [14]:
# ------------------- 2. View the Account table ------------------- #
print("\nViewing data from the Account table...")
query = "SELECT * FROM Account LIMIT 10;"
df_account = pd.read_sql(query, engine)
display(df_account)


Viewing data from the Account table...


,account_id,customer_id,date_of_opening,expected_incoming_activity,expected_outgoing_activity
0,ACC-1,C-1,1980-03-03,100000.0,10000.0
1,ACC-2,C-2,2010-01-03,200000.0,200000.0
2,ACC-3,C-2,2024-02-17,200000.0,200000.0
3,ACC-4,C-3,2024-09-03,2000.0,2000.0
4,ACC-5,C-4,2007-07-04,10000000.0,10000000.0


In [15]:
# ------------------- 3. View the Alert table ------------------- #
print("\nViewing data from the Alert table...")
query = "SELECT * FROM Alert LIMIT 10;"
df_alert = pd.read_sql(query, engine)
display(df_alert)



Viewing data from the Alert table...


,detection_id,alert_id,alert_date,customer_id,rule_name,alerted_transactions,false_positive_true_positive,alert_narrative
0,A-1-1,A-1,2024-10-03,C-1,Cash Structuring $10k,"T-1, T-2, T-3, T-4, T-5, T-6, T-7",True Positive,No reasonable explanation for customer activit...
1,A-1-2,A-1,2024-10-03,C-1,Cash Structuring $10k,"T-7, T-8, T-9, T-10, T-11, T-12",True Positive,NaN
2,A-1-3,A-1,2024-10-03,C-1,Rapid Movements of Funds,"T-1, T-2, T-3, T-4, T-5, T-6, T-7, T-8, T-9, T...",True Positive,NaN
3,A-1-4,A-1,2024-10-03,C-1,Large Wire to High Risk Jursidiction,T-13,True Positive,NaN
4,A-2-1,A-2,2024-10-03,C-2,Concentration Account,"T-14, T-15, T-16, T-17, T-20",True Positive,No reasonable explanation for customer activit...
5,A-2-2,A-2,2024-10-03,C-2,Rapid Movement of Funds,"T-14, T-15, T-16, T-17, T-20, T-21",True Positive,NaN
6,A-3-1,A-3,2024-10-03,C-3,New Account Rule,"T-22, T-23, T-29",False Positive,Customer explains he worked as a dealer for a ...
7,A-3-2,A-3,2024-10-03,C-3,Cash Structuring $10k,"T-22, T-23",False Positive,NaN
8,A-5-1,A-5,2024-10-03,C-4,Large Wire to High Risk Jursidiction,"T-30, T-31, T-32, T-33",True Positive,A-5-1: No reasonable explanation for moving fu...
9,A-5-2,A-5,2024-10-03,C-4,Large Wire to High Risk Jursidiction,T-34,False Positive,NaN


In [16]:
# ------------------- 4. View the Transaction table ------------------- #
print("\nViewing data of Transaction table...")
query = "SELECT * FROM Transaction LIMIT 10;"
df_transaction = pd.read_sql(query, engine)
display(df_transaction)

print("\nThe data of all tables has been successfully displayed!")



Viewing data of Transaction table...


,transaction_id,transaction_date,transaction_type,customer_id,account_id,incoming_outgoing,amount,originator,originator_country,beneficiary,beneficiary_country
0,T-1,2024-09-04,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
1,T-2,2024-09-05,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
2,T-3,2024-09-06,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
3,T-4,2024-09-07,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
4,T-5,2024-09-08,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
5,T-6,2024-09-09,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
6,T-7,2024-09-10,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
7,T-8,2024-09-11,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
8,T-9,2024-09-12,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
9,T-10,2024-09-13,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US



The data of all tables has been successfully displayed!


In [17]:
engine.dispose()
print("\nDatabase connection closed.")


Database connection closed.


# 2.Create RAG to improve local LLM generated SAR

In [18]:
!which python
!python -m pip --version

/bin/bash: line 1: python: command not found


In [19]:
import sys
print(sys.executable)


/usr/bin/python3


In [20]:
#!pip install --break-system-packages faiss-cpu langchain sentence-transformers
#!python3 -m pip install --break-system-packages faiss-cpu langchain sentence-transformers
!{sys.executable} -m pip install --break-system-packages faiss-cpu langchain sentence-transformers


Defaulting to user installation because normal site-packages is not writeable


### Create an engine to store four tables in PostgreSQL into FAISS (vector database)

In [21]:
import pandas as pd
from sqlalchemy import create_engine
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Connecting to PostgreSQL
db_user = "postgres"
db_password = "yourpassword"
db_host = "localhost"
db_name = "aml_database"

engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")

# Read all table data
df_customer = pd.read_sql("SELECT * FROM Customer", engine)
df_account = pd.read_sql("SELECT * FROM Account", engine)
df_alert = pd.read_sql("SELECT * FROM Alert", engine)
df_transaction = pd.read_sql("SELECT * FROM Transaction", engine)

print("All table data loading completed!")


All table data loading completed!


### Process data and convert to text format

In [22]:
# Define data formatting function
def format_customer(row):
    return f"Customer {row['customer_name']} ({row['customer_type']}) is engaged in {row['customer_line_of_business']}, expected products: {row['customer_expected_products']}, operating regions: {row['customer_expected_geographies']}. Registered country: {row['customer_incorporation_residence_country']}."

def format_account(row):
    return f"Account {row['account_id']} belongs to customer {row['customer_id']}, opened on {row['date_of_opening']}, expected incoming activity: {row['expected_incoming_activity']}, expected outgoing activity: {row['expected_outgoing_activity']}."

def format_alert(row):
    return f"Alert {row['alert_id']}, Detection ID: {row['detection_id']}, Customer {row['customer_id']}, triggered rule: {row['rule_name']}, involving transactions: {row['alerted_transactions']}. Alert details: {row['alert_narrative']}."

def format_transaction(row):
    return f"Transaction {row['transaction_id']}, type: {row['transaction_type']}, date: {row['transaction_date']}, amount: {row['amount']}, originator: {row['originator']} ({row['originator_country']}), beneficiary: {row['beneficiary']} ({row['beneficiary_country']})."

# Process data from all tables
customer_texts = df_customer.apply(format_customer, axis=1).tolist()
account_texts = df_account.apply(format_account, axis=1).tolist()
alert_texts = df_alert.apply(format_alert, axis=1).tolist()
transaction_texts = df_transaction.apply(format_transaction, axis=1).tolist()

# Merge All Text
all_texts = customer_texts + account_texts + alert_texts + transaction_texts
print(f"A total of {len(all_texts)} pieces of data need to be embedded")


A total of 48 pieces of data need to be embedded


### Generate embedding and save to FAISS

In [23]:
from sentence_transformers import SentenceTransformer

# Selecting an Embedding Model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate Embedding
embeddings = np.array([embedding_model.encode(text) for text in all_texts])

# Initialize FAISS
vector_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(vector_dim)

# Add data
index.add(embeddings)

print(f"FAISS vector database has been built, {index.ntotal} records stored!")


FAISS vector database has been built, 48 records stored!


## Implementing RAG Query: Use Data from Customer C-1 to Generate a SAR

### Retrieve Data for Customer C-1

In [24]:
# Retrieve data for customer C-1
customer_id = "C-1"

# Query database
df_c1_customer = df_customer[df_customer["customer_id"] == customer_id]
df_c1_accounts = df_account[df_account["customer_id"] == customer_id]
df_c1_alerts = df_alert[df_alert["customer_id"] == customer_id]
df_c1_transactions = df_transaction[df_transaction["customer_id"] == customer_id]

# Convert to text format
customer_text = df_c1_customer.apply(format_customer, axis=1).tolist()
account_texts = df_c1_accounts.apply(format_account, axis=1).tolist()
alert_texts = df_c1_alerts.apply(format_alert, axis=1).tolist()
transaction_texts = df_c1_transactions.apply(format_transaction, axis=1).tolist()

# Combine all data into one prompt
final_prompt = "\n".join(customer_text + account_texts + alert_texts + transaction_texts)

print("Generated Prompt for SAR:\n", final_prompt)


Generated Prompt for SAR:
 Customer John Diamond (Individual) is engaged in Manufacturing, expected products: ACH; Wire, operating regions: US. Registered country: US.
Account ACC-1 belongs to customer C-1, opened on 1980-03-03, expected incoming activity: 100000.0, expected outgoing activity: 10000.0.
Alert A-1, Detection ID: A-1-1, Customer C-1, triggered rule: Cash Structuring $10k, involving transactions: T-1, T-2, T-3, T-4, T-5, T-6, T-7. Alert details: No reasonable explanation for customer activity. Customer not in cash intensive business. No declared nexus with KY..
Alert A-1, Detection ID: A-1-2, Customer C-1, triggered rule: Cash Structuring $10k, involving transactions: T-7, T-8, T-9, T-10, T-11, T-12. Alert details: NaN.
Alert A-1, Detection ID: A-1-3, Customer C-1, triggered rule: Rapid Movements of Funds, involving transactions: T-1, T-2, T-3, T-4, T-5, T-6, T-7, T-8, T-9, T-10, T-11, T-12, T-13. Alert details: NaN.
Alert A-1, Detection ID: A-1-4, Customer C-1, triggered 

### Search FAISS for Relevant Context

In [25]:
import faiss
import numpy as np

# Convert the C-1 prompt into an embedding
query_embedding = embedding_model.encode(final_prompt).reshape(1, -1)

# Search FAISS for the top 5 most similar records
k = 5
distances, indices = index.search(query_embedding, k)

# Retrieve similar records from FAISS
similar_records = [all_texts[i] for i in indices[0]]

# Combine similar records for context
context_text = "\n".join(similar_records)

print("Retrieved Similar Cases for SAR Generation:\n", context_text)


Retrieved Similar Cases for SAR Generation:
 Alert A-1, Detection ID: A-1-1, Customer C-1, triggered rule: Cash Structuring $10k, involving transactions: T-1, T-2, T-3, T-4, T-5, T-6, T-7. Alert details: No reasonable explanation for customer activity. Customer not in cash intensive business. No declared nexus with KY..
Alert A-1, Detection ID: A-1-2, Customer C-1, triggered rule: Cash Structuring $10k, involving transactions: T-7, T-8, T-9, T-10, T-11, T-12. Alert details: NaN.
Alert A-3, Detection ID: A-3-2, Customer C-3, triggered rule: Cash Structuring $10k, involving transactions: T-22, T-23. Alert details: NaN.
Alert A-2, Detection ID: A-2-1, Customer C-2, triggered rule: Concentration Account, involving transactions: T-14, T-15, T-16, T-17, T-20. Alert details: No reasonable explanation for customer activity. Round dollar transaction unusual in normal course of business..
Alert A-5, Detection ID: A-5-1, Customer C-4, triggered rule: Large Wire to High Risk Jursidiction, involvin

### Generate the SAR Report using DeepSeek

In [26]:
#!pip install boto3
#!sudo apt install -y boto3
!sudo apt install -y python3-boto3

Reading package lists... Done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Building dependency tree... Done
Reading state information... Done
python3-boto3 is already the newest version (1.34.46+dfsg-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.


In [27]:
import boto3
import json

# Connect to AWS Bedrock
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Build the prompt generated by SAR
sar_generation_prompt = f"""
Generate a Suspicious Activity Report (SAR) based on the following data:

### Customer Information
{final_prompt}

### Similar Cases for Context
{context_text}

The SAR should include:
- A clear narrative of suspicious activity
- Relevant transactions and patterns
- Possible justifications for suspicion

Please generate a professional and structured SAR report.
"""

# Calling the LLaMA 3 70B model
response = client.invoke_model(
    modelId="us.meta.llama3-1-70b-instruct-v1:0",
    body=json.dumps({
        "prompt": sar_generation_prompt,
        "temperature": 0.7,  # Controlling generation randomness
        "top_p": 0.9
    })
)

# Parsing the returned JSON
response_body = json.loads(response["body"].read().decode("utf-8"))

# Printing SAR Reports
print("\nGenerated SAR Report:\n")
print(response_body["generation"])



Generated SAR Report:

---

**Suspicious Activity Report (SAR)**

**Date:** 2024-09-17

**Customer Information:**

- **Name:** John Diamond
- **Account Number:** ACC-1
- **Customer Type:** Individual
- **Business Type:** Manufacturing
- **Expected Products:** ACH, Wire
- **Operating Regions:** US
- **Registered Country:** US

**Summary of Suspicious Activity:**

John Diamond, an individual customer engaged in manufacturing, has been identified as exhibiting suspicious activity through multiple transactions and alerts. The activity suggests potential structuring of cash deposits to avoid reporting thresholds, rapid movements of funds, and a large wire transfer to a high-risk jurisdiction.

**Relevant Transactions and Patterns:**

1. **Cash Structuring $10k:** Between 2024-09-04 and 2024-09-15, John Diamond made 12 cash deposits (T-1 through T-12), each amounting to $9,000. This pattern suggests an attempt to structure transactions below the $10,000 reporting threshold.
2. **Rapid Movem

In [32]:
import boto3
import json

# Connect to AWS Bedrock
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Build the prompt generated by SAR
sar_generation_prompt = f"""
Generate a Suspicious Activity Report (SAR) based on the following data:

### Customer Information
{final_prompt}

### Similar Cases for Context
{context_text}

The SAR should include:
- A clear narrative of suspicious activity
- Relevant transactions and patterns
- Possible justifications for suspicion

Please generate a professional and structured SAR report.
"""

# Calling the LLaMA 3 70B model
response = client.invoke_model(
    modelId="us.deepseek.r1-v1:0",
    body=json.dumps({
        "prompt": sar_generation_prompt,
        "temperature": 0.7,  # Controlling generation randomness
        "top_p": 0.9
    })
)

# Parsing the returned JSON
response_body = json.loads(response["body"].read().decode("utf-8"))
generated_text = response_body.get("choices", [{}])[0].get("text", "No response generated.")

# Printing SAR Reports
print("\nGenerated SAR Report:\n")
print(generated_text)



Generated SAR Report:

</think>

**Suspicious Activity Report (SAR)**  
**Filing Institution:** [Insert Institution Name]  
**Filing Date:** [Insert Date]  
**SAR Narrative Reference Number:** [Insert Reference Number]  

---

### **Subject Information**  
- **Customer Name:** John Diamond (Individual)  
- **Account Number:** ACC-1 (Opened: March 3, 1980)  
- **Business Type:** Manufacturing (Non-cash-intensive industry)  
- **Jurisdiction:** Registered in the U.S., operating primarily in the U.S.  

---

### **Summary of Suspicious Activity**  
The customer, John Diamond, engaged in a series of structured cash deposits and a large wire transfer to a high-risk jurisdiction (Cayman Islands) inconsistent with their declared business profile. The activity triggered four alerts:  
1. **Cash Structuring ($10k Threshold)** across two detection sequences (A-1-1 and A-1-2).  
2. **Rapid Movement of Funds** (A-1-3).  
3. **Large Wire to High-Risk Jurisdiction** (A-1-4).  

---

### **Detailed 

In [33]:
engine.dispose()
print("\nDatabase connection closed.")


Database connection closed.


In [34]:
import boto3
import json

client = boto3.client("bedrock", region_name="us-east-1")

# 获取所有可用的模型信息
response = client.list_foundation_models()

# 查找 DeepSeek 相关的模型
deepseek_models = [model for model in response["modelSummaries"] if "deepseek" in model["modelId"]]

# 打印 DeepSeek 相关模型信息
print(json.dumps(deepseek_models, indent=2))


[
  {
    "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/deepseek.r1-v1:0",
    "modelId": "deepseek.r1-v1:0",
    "modelName": "DeepSeek-R1",
    "providerName": "DeepSeek",
    "inputModalities": [
      "TEXT"
    ],
    "outputModalities": [
      "TEXT"
    ],
    "responseStreamingSupported": true,
    "customizationsSupported": [],
    "inferenceTypesSupported": [
      "INFERENCE_PROFILE"
    ],
    "modelLifecycle": {
      "status": "ACTIVE"
    }
  }
]


# Newest code

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

In [2]:
# Connecting to PostgreSQL
db_user = "postgres"
db_password = "yourpassword"
db_host = "localhost"
db_name = "aml_database"

engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")


In [3]:
# Load all table data
df_customer = pd.read_sql("SELECT * FROM Customer", engine)
df_account = pd.read_sql("SELECT * FROM Account", engine)
df_alert = pd.read_sql("SELECT * FROM Alert", engine)
df_transaction = pd.read_sql("SELECT * FROM Transaction", engine)

print("✅ All table data loaded successfully!")

✅ All table data loaded successfully!


In [4]:
# Load FAISS index
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
vector_dim = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(vector_dim)

# Convert all text data into embeddings
def format_data():
    customer_texts = df_customer.apply(lambda row: f"Customer {row['customer_name']} ({row['customer_type']}) engaged in {row['customer_line_of_business']}", axis=1).tolist()
    account_texts = df_account.apply(lambda row: f"Account {row['account_id']} belongs to {row['customer_id']}", axis=1).tolist()
    alert_texts = df_alert.apply(lambda row: f"Alert {row['alert_id']}, rule: {row['rule_name']}", axis=1).tolist()
    transaction_texts = df_transaction.apply(lambda row: f"Transaction {row['transaction_id']} amount: {row['amount']}", axis=1).tolist()
    return customer_texts + account_texts + alert_texts + transaction_texts

all_texts = format_data()
embeddings = np.array([embedding_model.encode(text) for text in all_texts])
index.add(embeddings)

print(f"✅ FAISS vector database has been initialized with {index.ntotal} records!")

✅ FAISS vector database has been initialized with 48 records!


In [5]:
# Retrieve relevant data for SAR generation
def retrieve_sar_data(customer_id):
    customer = df_customer[df_customer['customer_id'] == customer_id].iloc[0]
    accounts = df_account[df_account['customer_id'] == customer_id]
    alerts = df_alert[df_alert['customer_id'] == customer_id]
    transactions = df_transaction[df_transaction['customer_id'] == customer_id]
    
    # Retrieve similar cases from FAISS
    query_embedding = embedding_model.encode(f"Customer {customer['customer_name']}")
    _, similar_case_indices = index.search(np.array([query_embedding]), 3)  # Top 3 matches
    similar_cases = [all_texts[i] for i in similar_case_indices[0]]
    
    return customer, accounts, alerts, transactions, similar_cases

In [14]:
def generate_sar_report(customer_id):
    # 获取数据
    customer, accounts, alerts, transactions, similar_cases = retrieve_sar_data(customer_id)
    
    # 提取警报信息
    if not alerts.empty:
        alert_id = alerts.iloc[0]["alert_id"]
        alert_date = alerts.iloc[0]["alert_date"]
        rule_name_1 = alerts.iloc[0]["rule_name"]
        alert_narrative_1 = alerts.iloc[0]["alert_narrative"]
        high_risk_country = "Unknown"  # 这里可以进一步定义
    else:
        alert_id = "N/A"
        alert_date = "N/A"
        rule_name_1 = "N/A"
        alert_narrative_1 = "N/A"
        high_risk_country = "N/A"

    # 提取交易信息
    transaction_count = len(transactions)
    total_amount = transactions["amount"].sum() if not transactions.empty else 0
    transaction_details = "\n".join([
        f"- {row['transaction_date']}: {row['transaction_type']} of ${row['amount']} from {row['originator']} ({row['originator_country']}) to {row['beneficiary']} ({row['beneficiary_country']})"
        for _, row in transactions.iterrows()
    ])

    # 提取账户信息
    if not accounts.empty:
        account_id = accounts.iloc[0]["account_id"]
    else:
        account_id = "N/A"

    # 组装 SAR 报告
    sar_report = f"""
### **Suspicious Activity Report (SAR)**

**Institution:** LLM Bank New York Branch ("LLM NY")  
**Internal SAR Reference Number:** {alert_id}  
**Report Date:** {alert_date}  

---

#### **1️⃣ Transaction Summary**
Between {transactions['transaction_date'].min() if not transactions.empty else 'N/A'} and {transactions['transaction_date'].max() if not transactions.empty else 'N/A'}, {customer['customer_name']} ("{customer_id}") ({customer['customer_incorporation_residence_country']}) conducted {transaction_count} transactions totaling ${total_amount}. These transactions were processed through {account_id} (the "Subject Account") at LLM NY.

Key transactions:
{transaction_details if transaction_details else "N/A"}

---

#### **2️⃣ Customer Profile**
**Name:** {customer['customer_name']}  
**DOB:** {customer.get('dob', 'N/A')}  
**SSN:** {customer.get('ssn', 'N/A')}  
**Address:** {customer.get('address', 'N/A')}  
**Occupation:** {customer['customer_line_of_business']}  
**Business Relationship:** {customer['customer_expected_products']}, {customer['customer_expected_geographies']}  
**Country of Incorporation/Residence:** {customer['customer_incorporation_residence_country']}  

---

#### **3️⃣ Suspicious Activity Indicators**
The following indicators contributed to this SAR filing:
1. **{rule_name_1}**: {alert_narrative_1}
2. **High-Risk Jurisdiction Involvement**: {high_risk_country}

---

#### **4️⃣ External Research**
External research was unable to conclusively identify a legitimate line of business for the counterparty {transactions.iloc[-1]['beneficiary'] if not transactions.empty else "N/A"} ({transactions.iloc[-1]['beneficiary_country'] if not transactions.empty else "N/A"}).

---

#### **5️⃣ Conclusion**
This SAR is filed due to:
- Possible structuring of cash deposits
- Large wire transfers to offshore accounts
- Transactions with no apparent economic or business purpose

---

#### **6️⃣ Contact Information**
This SAR pertains to LLM NY Case No. {alert_id}. For inquiries, please contact:
- **Donald J. Orange**, Chief Compliance Officer (646-555-5555, donaldjorange@llmbank.com)
- **Alyn Mask**, General Counsel (646-666-6666, alynmask@llmbank.com)

All supporting documentation is maintained by the Financial Crime Compliance Department at LLM NY.
"""

    return sar_report

# 示例执行
customer_id = "C-1"
print(generate_sar_report(customer_id))



### **Suspicious Activity Report (SAR)**

**Institution:** LLM Bank New York Branch ("LLM NY")  
**Internal SAR Reference Number:** A-1  
**Report Date:** 2024-10-03  

---

#### **1️⃣ Transaction Summary**
Between 2024-09-04 and 2024-09-16, John Diamond ("C-1") (US) conducted 13 transactions totaling $213000.0. These transactions were processed through ACC-1 (the "Subject Account") at LLM NY.

Key transactions:
- 2024-09-04: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-05: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-06: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-07: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-08: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-09: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-10: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09

In [12]:
def generate_sar_report(customer_id):
    # 获取数据
    customer, accounts, alerts, transactions, similar_cases = retrieve_sar_data(customer_id)
    
    # 提取警报信息
    if not alerts.empty:
        alert_id = alerts.iloc[0]["alert_id"]
        alert_date = alerts.iloc[0]["alert_date"]
        rule_name_1 = alerts.iloc[0]["rule_name"]
        alert_narrative_1 = alerts.iloc[0]["alert_narrative"]
        high_risk_country = "Unknown"  # 这里可以进一步定义
    else:
        alert_id = "N/A"
        alert_date = "N/A"
        rule_name_1 = "N/A"
        alert_narrative_1 = "N/A"
        high_risk_country = "N/A"

    # 提取交易信息
    transaction_count = len(transactions)
    total_amount = transactions["amount"].sum() if not transactions.empty else 0
    transaction_details = "\n".join([
        f"- {row['transaction_date']}: {row['transaction_type']} of ${row['amount']} from {row['originator']} ({row['originator_country']}) to {row['beneficiary']} ({row['beneficiary_country']})"
        for _, row in transactions.iterrows()
    ])

    # 提取账户信息
    if not accounts.empty:
        account_id = accounts.iloc[0]["account_id"]
    else:
        account_id = "N/A"

    # 组装 SAR 报告
    sar_report = f"""
### **Suspicious Activity Report (SAR)**

LLM Bank New York Branch ("LLM NY") is a wholesale branch of LLM Bank Ltd. ("LLM"), a commercial bank located in mainland China. 
**Internal SAR Reference Number:** {alert_id}  
**Report Date:** {alert_date}  

---

#### **1️⃣ Transaction Summary**
Between {transactions['transaction_date'].min() if not transactions.empty else 'N/A'} and {transactions['transaction_date'].max() if not transactions.empty else 'N/A'}, {customer['customer_name']} ("{customer_id}") ({customer['customer_incorporation_residence_country']}) conducted {transaction_count} transactions totaling ${total_amount}. These transactions were processed through {account_id} (the "Subject Account") at LLM NY.

Key transactions:
{transaction_details if transaction_details else "N/A"}

---

#### **2️⃣ Customer Profile**
**Name:** {customer['customer_name']}  
**DOB:** {customer.get('dob', 'N/A')}  
**SSN:** {customer.get('ssn', 'N/A')}  
**Address:** {customer.get('address', 'N/A')}  
**Occupation:** {customer['customer_line_of_business']}  
**Business Relationship:** {customer['customer_expected_products']}, {customer['customer_expected_geographies']}  
**Country of Incorporation/Residence:** {customer['customer_incorporation_residence_country']}  

---

#### **3️⃣ Suspicious Activity Indicators**
The following indicators contributed to this SAR filing:
1. **{rule_name_1}**: {alert_narrative_1}
2. **High-Risk Jurisdiction Involvement**: {high_risk_country}

---

#### **4️⃣ External Research**
External research was unable to conclusively identify a legitimate line of business for the counterparty {transactions.iloc[-1]['beneficiary'] if not transactions.empty else "N/A"} ({transactions.iloc[-1]['beneficiary_country'] if not transactions.empty else "N/A"}).

---

#### **5️⃣ Conclusion**
This SAR is filed due to:
- Possible structuring of cash deposits
- Large wire transfers to offshore accounts
- Transactions with no apparent economic or business purpose

---

#### **6️⃣ Contact Information**
This SAR pertains to LLM NY Case No. {alert_id}. For inquiries, please contact:
- **Donald J. Orange**, Chief Compliance Officer (646-555-5555, donaldjorange@llmbank.com)
- **Alyn Mask**, General Counsel (646-666-6666, alynmask@llmbank.com)

All supporting documentation is maintained by the Financial Crime Compliance Department at LLM NY.
"""

    return sar_report

# 示例执行
customer_id = "C-1"
print(generate_sar_report(customer_id))



### **Suspicious Activity Report (SAR)**

LLM Bank New York Branch ("LLM NY") is a wholesale branch of LLM Bank Ltd. ("LLM"), a commercial bank located in mainland China. 
**Internal SAR Reference Number:** A-1  
**Report Date:** 2024-10-03  

---

#### **1️⃣ Transaction Summary**
Between 2024-09-04 and 2024-09-16, John Diamond ("C-1") (US) conducted 13 transactions totaling $213000.0. These transactions were processed through ACC-1 (the "Subject Account") at LLM NY.

Key transactions:
- 2024-09-04: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-05: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-06: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-07: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-08: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-09: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
- 2024-09-10: Ca

In [15]:
def generate_sar_report(customer_id):
    # 获取数据
    customer, accounts, alerts, transactions, similar_cases = retrieve_sar_data(customer_id)
    
    # 提取警报信息
    if not alerts.empty:
        alert_id = alerts.iloc[0]["alert_id"]
        alert_date = alerts.iloc[0]["alert_date"]
        rule_name_1 = alerts.iloc[0]["rule_name"]
        alert_narrative_1 = alerts.iloc[0]["alert_narrative"]
        high_risk_country = "Unknown"  # 这里可以进一步定义
    else:
        alert_id = "N/A"
        alert_date = "N/A"
        rule_name_1 = "N/A"
        alert_narrative_1 = "N/A"
        high_risk_country = "N/A"

    # 提取交易信息
    transaction_count = len(transactions)
    total_amount = transactions["amount"].sum() if not transactions.empty else 0
    transaction_details = "\n".join([
        f"{row['transaction_date']}: {row['transaction_type']} of ${row['amount']} from {row['originator']} ({row['originator_country']}) to {row['beneficiary']} ({row['beneficiary_country']})"
        for _, row in transactions.iterrows()
    ])

    # 提取账户信息
    if not accounts.empty:
        account_id = accounts.iloc[0]["account_id"]
    else:
        account_id = "N/A"

    # 生成更正式的 SAR 报告
    sar_report = f"""LLM Bank New York Branch ("LLM NY") is a wholesale branch of LLM Bank Ltd. ("LLM"), a commercial bank located in mainland China. LLM NY is filing this Suspicious Activity Report ("SAR") (Internal SAR Reference Number {alert_id}) to report {transaction_count} transactions totaling ${total_amount} and sent between {transactions['transaction_date'].min() if not transactions.empty else 'N/A'} and {transactions['transaction_date'].max() if not transactions.empty else 'N/A'}.

Between {transactions['transaction_date'].min() if not transactions.empty else 'N/A'} and {transactions['transaction_date'].max() if not transactions.empty else 'N/A'}, {customer['customer_name']} ("{customer_id}") ({customer['customer_incorporation_residence_country']}) conducted {transaction_count} transactions totaling ${total_amount}. These transactions were processed through {account_id} ("the Subject Account") at LLM NY.

Key transactions:
{transaction_details if transaction_details else "N/A"}

Internal LLM NY KYC information identified {customer['customer_name']} with the following details: DOB: {customer.get('dob', 'N/A')}; SSN: {customer.get('ssn', 'N/A')}; Address: {customer.get('address', 'N/A')}; and Occupation: {customer['customer_line_of_business']}. There is no apparent connection between {customer['customer_name']} and {transactions.iloc[-1]['beneficiary'] if not transactions.empty else "N/A"} ({transactions.iloc[-1]['beneficiary_country'] if not transactions.empty else "N/A"}).

External research was unable to conclusively identify a legitimate line of business for {transactions.iloc[-1]['beneficiary'] if not transactions.empty else "N/A"}.

These transactions are being reported due to the following: (1) {rule_name_1}: {alert_narrative_1}; (2) High-Risk Jurisdiction Involvement: {high_risk_country}.

This SAR pertains to LLM NY Case No. {alert_id}. For inquiries, please contact Donald J. Orange, Chief Compliance Officer and Chief BSA/AML Officer (646-555-5555 or donaldjorange@llmbank.com) or Alyn Mask, General Counsel (646-666-6666 or alynmask@llmbank.com). All supporting documentation is maintained by the Financial Crime Compliance Department at LLM NY."""

    return sar_report

# 示例执行
customer_id = "C-1"
print(generate_sar_report(customer_id))


LLM Bank New York Branch ("LLM NY") is a wholesale branch of LLM Bank Ltd. ("LLM"), a commercial bank located in mainland China. LLM NY is filing this Suspicious Activity Report ("SAR") (Internal SAR Reference Number A-1) to report 13 transactions totaling $213000.0 and sent between 2024-09-04 and 2024-09-16.

Between 2024-09-04 and 2024-09-16, John Diamond ("C-1") (US) conducted 13 transactions totaling $213000.0. These transactions were processed through ACC-1 ("the Subject Account") at LLM NY.

Key transactions:
2024-09-04: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-05: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-06: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-07: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-08: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-09: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)

In [16]:
def generate_sar_report(customer_id):
    # 获取数据
    customer, accounts, alerts, transactions, similar_cases = retrieve_sar_data(customer_id)
    
    # 提取警报信息
    if not alerts.empty:
        alert_id = alerts.iloc[0]["alert_id"]
        alert_date = alerts.iloc[0]["alert_date"]
        rule_name_1 = alerts.iloc[0]["rule_name"]
        alert_narrative_1 = alerts.iloc[0]["alert_narrative"]
        high_risk_country = "Unknown"  # 这里可以进一步定义
    else:
        alert_id = "N/A"
        alert_date = "N/A"
        rule_name_1 = "N/A"
        alert_narrative_1 = "N/A"
        high_risk_country = "N/A"

    # 提取交易信息
    transaction_count = len(transactions)
    total_amount = transactions["amount"].sum() if not transactions.empty else 0
    transaction_details = "\n".join([
        f"{row['transaction_date']}: {row['transaction_type']} of ${row['amount']} from {row['originator']} ({row['originator_country']}) to {row['beneficiary']} ({row['beneficiary_country']})"
        for _, row in transactions.iterrows()
    ])

    # 提取账户信息
    if not accounts.empty:
        account_id = accounts.iloc[0]["account_id"]
    else:
        account_id = "N/A"

    # **✨ 调用 FAISS 召回相似案例 ✨**
    query_embedding = embedding_model.encode(f"Customer {customer['customer_name']} - {customer['customer_line_of_business']}")
    _, similar_case_indices = index.search(np.array([query_embedding]), 3)  # 召回 Top 3 相似案例
    similar_cases_text = "\n".join([all_texts[i] for i in similar_case_indices[0]])  # 获取相似案例文本
    
    # 生成 SAR 报告
    sar_report = f"""LLM Bank New York Branch ("LLM NY") is a wholesale branch of LLM Bank Ltd. ("LLM"), a commercial bank located in mainland China. LLM NY is filing this Suspicious Activity Report ("SAR") (Internal SAR Reference Number {alert_id}) to report {transaction_count} transactions totaling ${total_amount} and sent between {transactions['transaction_date'].min() if not transactions.empty else 'N/A'} and {transactions['transaction_date'].max() if not transactions.empty else 'N/A'}.

Between {transactions['transaction_date'].min() if not transactions.empty else 'N/A'} and {transactions['transaction_date'].max() if not transactions.empty else 'N/A'}, {customer['customer_name']} ("{customer_id}") ({customer['customer_incorporation_residence_country']}) conducted {transaction_count} transactions totaling ${total_amount}. These transactions were processed through {account_id} ("the Subject Account") at LLM NY.

Key transactions:
{transaction_details if transaction_details else "N/A"}

Internal LLM NY KYC information identified {customer['customer_name']} with the following details: DOB: {customer.get('dob', 'N/A')}; SSN: {customer.get('ssn', 'N/A')}; Address: {customer.get('address', 'N/A')}; and Occupation: {customer['customer_line_of_business']}. There is no apparent connection between {customer['customer_name']} and {transactions.iloc[-1]['beneficiary'] if not transactions.empty else "N/A"} ({transactions.iloc[-1]['beneficiary_country'] if not transactions.empty else "N/A"}).

External research was unable to conclusively identify a legitimate line of business for {transactions.iloc[-1]['beneficiary'] if not transactions.empty else "N/A"}.

These transactions are being reported due to the following: (1) {rule_name_1}: {alert_narrative_1}; (2) High-Risk Jurisdiction Involvement: {high_risk_country}.

**Similar Cases Retrieved from FAISS:**
{similar_cases_text}

This SAR pertains to LLM NY Case No. {alert_id}. For inquiries, please contact Donald J. Orange, Chief Compliance Officer and Chief BSA/AML Officer (646-555-5555 or donaldjorange@llmbank.com) or Alyn Mask, General Counsel (646-666-6666 or alynmask@llmbank.com). All supporting documentation is maintained by the Financial Crime Compliance Department at LLM NY."""

    return sar_report

# 示例执行
customer_id = "C-1"
print(generate_sar_report(customer_id))


LLM Bank New York Branch ("LLM NY") is a wholesale branch of LLM Bank Ltd. ("LLM"), a commercial bank located in mainland China. LLM NY is filing this Suspicious Activity Report ("SAR") (Internal SAR Reference Number A-1) to report 13 transactions totaling $213000.0 and sent between 2024-09-04 and 2024-09-16.

Between 2024-09-04 and 2024-09-16, John Diamond ("C-1") (US) conducted 13 transactions totaling $213000.0. These transactions were processed through ACC-1 ("the Subject Account") at LLM NY.

Key transactions:
2024-09-04: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-05: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-06: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-07: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-08: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)
2024-09-09: Cash Deposit of $9000.0 from John Diamond (US) to John Diamond (US)